In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow.keras as keras

ModuleNotFoundError: No module named 'tensorflow'

### EDA - MNIST Dataset

### Data Preparation

### CNN

### Conclusions and Insights